#### Section 1.4.4 Linear Regulator
Taken from "Switch-Mode Power Supplies" by Christophe P. Basso

In [1]:
# Project selection

MY_PROJECT: str = "sec_1_04_04"  # which section of config file to use

In [2]:
# Imports, type aliases

import copy
import tomllib
from pathlib import Path
from typing import Any, TypeAlias

import numpy as np
import numpy.typing as npt
import py4spice as spi

# Aliases for type checking
numpy_flt: TypeAlias = npt.NDArray[np.float64]

In [3]:
# Constants


class Key:
    """Keys for dictionaries.  Defined here at top level so they can be
    referenced instead of using strings for keys.
    """

    # Keys for decoding the config file
    CONFIG_NAME = "config_name"
    GLOBAL_SECTION = "global_section"
    NGSPICE_EXE_KEY = "ngspice_exe_str"
    NETLISTS_DIR_KEY = "netlists_dir_str"
    RESULTS_DIR_KEY = "results_dir_str"
    SIM_TRANSCRIPT_KEY = "sim_transcript_str"
    PROJ_PATH_KEY = "proj_path_str"
    PROJ_SECTION = "proj_section"

    # Keys for the paths_dict
    NGSPICE_EXE = "ngspice_exe"
    PROJ_PATH = "proj_path"
    NETLISTS_PATH = "netlists_path"
    RESULTS_PATH = "results_path"
    SIM_TRANSCRIPT_FILENAME = "sim_transcript_filename"

    # Keys for the netlists_dict
    BLANKLINE = "blankline"
    TITLE = "title"
    END_LINE = "end_line"
    LOAD1 = "load1"
    LOAD2 = "load2"
    LOAD3 = "load3"
    STIMULUS1 = "stimulus1"
    STIMULUS2 = "stimulus2"
    STIMULUS3 = "stimulus3"
    SUPPLIES = "supplies"
    MODELS = "models"
    DUT = "dut"
    CONTROL1 = "control1"
    CONTROL2 = "control2"
    TOP1 = "top1"
    TOP2 = "top2"

    # Keys for the vectors_dict
    VEC_ALL = "vec_all"
    VEC_IN_OUT = "vec_in_out"
    VEC_OUT = "vec_out"


# Dictionary for decoding the config file. These keys match the name is the config file
# DO NOT CHANGE unless you change the keys in the config file
config_file_decoding = {
    Key.CONFIG_NAME: "/workspaces/sw_pwr_book_sim/python/config.toml",
    Key.GLOBAL_SECTION: "global",
    Key.PROJ_SECTION: MY_PROJECT,
    Key.NGSPICE_EXE_KEY: "ngspice_exe_str",
    Key.NETLISTS_DIR_KEY: "netlists_dir_str",
    Key.RESULTS_DIR_KEY: "results_dir_str",
    Key.SIM_TRANSCRIPT_KEY: "sim_transcript_str",
    Key.PROJ_PATH_KEY: "proj_path_str",
}

In [4]:
# Initialization

# read config file and create config dictionary
config_name: Path = Path(config_file_decoding[Key.CONFIG_NAME])
with open(config_name, "rb") as file:
    my_config: dict[str, Any] = tomllib.load(file)

# Decodings for the config dictionary, which came from the config file
config_global_section: str = config_file_decoding[Key.GLOBAL_SECTION]
config_ngspice_exe_key: str = config_file_decoding[Key.NGSPICE_EXE_KEY]
config_netlists_dir_key: str = config_file_decoding[Key.NETLISTS_DIR_KEY]
config_results_dir_key: str = config_file_decoding[Key.RESULTS_DIR_KEY]
config_sim_transcript_key: str = config_file_decoding[Key.SIM_TRANSCRIPT_KEY]
config_proj_path_key: str = config_file_decoding[Key.PROJ_PATH_KEY]
config_proj_section: str = config_file_decoding[Key.PROJ_SECTION]

# Create paths based on the config dictionary
ngspice_exe: Path = Path(my_config[config_global_section][config_ngspice_exe_key])
proj_path: Path = Path(my_config[config_proj_section][config_proj_path_key])
netlists_path: Path = (
    proj_path / my_config[config_global_section][config_netlists_dir_key]
)
results_path: Path = (
    proj_path / my_config[config_global_section][config_results_dir_key]
)

# create results directory if it does not exist
results_path.mkdir(parents=True, exist_ok=True)

# create simlulation transcript file. If it exists, make sure it is empty
sim_tran_filename: Path = (
    results_path / my_config[config_global_section][config_sim_transcript_key]
)
if sim_tran_filename.exists():  # delete and recreate. this makes sure it's empty
    sim_tran_filename.unlink()
sim_tran_filename.touch()

# create paths dictionary
paths_dict = {
    Key.NGSPICE_EXE: ngspice_exe,
    Key.PROJ_PATH: proj_path,
    Key.NETLISTS_PATH: netlists_path,
    Key.RESULTS_PATH: results_path,
    Key.SIM_TRANSCRIPT_FILENAME: sim_tran_filename,
}

# netlists_dict = define_netlists(paths_dict)
netlists_path: Path = paths_dict[Key.NETLISTS_PATH]  # make shorter alias
netlists_dict: dict[str, spi.Netlist] = {}  # create empty netlist dictionary

netlists_dict[Key.BLANKLINE] = spi.Netlist("")  # blank line for spacing
netlists_dict[Key.TITLE] = spi.Netlist("* Title line")  # title line
netlists_dict[Key.END_LINE] = spi.Netlist(".end")  # end statement

# create netlist objects from files and add to netlist dictionary
netlists_dict[Key.DUT] = spi.Netlist(netlists_path / "dut.cir")
netlists_dict[Key.LOAD1] = spi.Netlist(netlists_path / "load_resistive.cir")
netlists_dict[Key.LOAD2] = spi.Netlist(netlists_path / "load_resistive.cir")
netlists_dict[Key.LOAD3] = spi.Netlist(netlists_path / "load_current_pulse.cir")
netlists_dict[Key.STIMULUS1] = spi.Netlist(netlists_path / "stimulus_15v_dc.cir")
netlists_dict[Key.STIMULUS2] = spi.Netlist(netlists_path / "stimulus_15v_ramp.cir")
netlists_dict[Key.STIMULUS3] = spi.Netlist(netlists_path / "stimulus_15v_dc.cir")
netlists_dict[Key.SUPPLIES] = spi.Netlist(netlists_path / "supplies.cir")
netlists_dict[Key.MODELS] = spi.Netlist(netlists_path / "models.cir")

# Define a vector dictionary for simulation and post-simulation analysis
vectors_dict = {
    Key.VEC_ALL: spi.Vectors("all"),
    Key.VEC_IN_OUT: spi.Vectors("in out"),
    Key.VEC_OUT: spi.Vectors("out"),
}

#### Part 1: do something
Say some stuff.

In [5]:
# Sim1

# Define analyses
list_of_analyses: list[spi.Analyses] = []  # start with an empty list

# 1st analysis: operating point
op1 = spi.Analyses(
    name="op1",
    cmd_type="op",
    cmd="op",
    vector=vectors_dict[Key.VEC_ALL],
    results_loc=paths_dict[Key.RESULTS_PATH],
)
list_of_analyses.append(op1)

# 2nd analysis: transfer function
tf1 = spi.Analyses(
    name="tf1",
    cmd_type="tf",
    cmd="tf v(out) vin",
    vector=vectors_dict[Key.VEC_ALL],
    results_loc=paths_dict[Key.RESULTS_PATH],
)
list_of_analyses.append(tf1)

# create control section
my_control = spi.Control()  # create 'my_control' object
for analysis in list_of_analyses:
    my_control.insert_lines(analysis.lines_for_cntl())
netlists_dict[Key.CONTROL1] = spi.Netlist(str(my_control))

# concatenate all tne netlists to make top1 and add to netlist dict
netlists_dict[Key.TOP1] = (
    netlists_dict[Key.TITLE]
    + netlists_dict[Key.BLANKLINE]
    + netlists_dict[Key.DUT]
    + netlists_dict[Key.LOAD1]
    + netlists_dict[Key.BLANKLINE]
    + netlists_dict[Key.SUPPLIES]
    + netlists_dict[Key.BLANKLINE]
    + netlists_dict[Key.STIMULUS1]
    + netlists_dict[Key.BLANKLINE]
    + netlists_dict[Key.MODELS]
    + netlists_dict[Key.BLANKLINE]
    + netlists_dict[Key.CONTROL1]
    + netlists_dict[Key.END_LINE]
)
# write netlist to a file so ngspice can read it
top1_filename: Path = paths_dict[Key.NETLISTS_PATH] / "top1.cir"
netlists_dict[Key.TOP1].write_to_file(top1_filename)

# prepare simulate object, print out command, and simulate
sim1: spi.Simulate = spi.Simulate(
    paths_dict[Key.NGSPICE_EXE],
    top1_filename,
    paths_dict[Key.SIM_TRANSCRIPT_FILENAME],
    "sim1",
)
# spi.print_section("Ngspice Command", sim1) # print out command
sim1.run()  # run the Ngspice simulation

# convert the raw results into list of SimResults objects
sim_results: list[spi.SimResults] = [
    spi.SimResults.from_file(analysis.cmd_type, analysis.results_filename)
    for analysis in list_of_analyses
]

# give each SimResults object a more descriptive name
op1_results, tf1_results = sim_results

# diaplay results for operating point analysis
spi.print_section("Operating Point Results", op1_results.print_table())

# display results for small signal transfer function analysis
spi.print_section("Part 1: Transfer Function Results", tf1_results.print_table())


--- Operating Point Results ---
b.x3.b1#branch  -0.100076
beta             0.4341393
com              0.0
div              2.495659
e.x2.e1#branch   0.0
e1#branch        0.0
e2#branch        0.0
g                4.341393
gain             0.75
in               15.0
out              4.991317
out_meas         0.0
sum              5.091393
vcom#branch      0.0002495659
vin#branch       0.0
vmeas#branch     0.09982634
vref             2.5
vref#branch      0.0
-------------------------------


--- Part 1: Transfer Function Results ---
transfer_function            9.979641e-05
output_impedance_at_v(out)   0.001995928
vin#input_impedance          1e+20
-----------------------------------------



#### Part 2: do something
Say some stuff.

In [8]:
# Sim2

# Define analyses
list_of_analyses: list[spi.Analyses] = []  # start with an empty list
# 1st (and only) analysis: transient analysis
tr1 = spi.Analyses(
    name="tr1",
    cmd_type="",
    cmd="tran 1e-9 20e-6",
    vector=vectors_dict[Key.VEC_ALL],
    results_loc=paths_dict[Key.RESULTS_PATH],
)
list_of_analyses.append(tr1)

# create control section
my_control = spi.Control()  # create 'my_control' object
for analysis in list_of_analyses:
    my_control.insert_lines(analysis.lines_for_cntl())
netlists_dict[Key.CONTROL2] = spi.Netlist(str(my_control))

# concatenate all tne netlists to make top1 and add to netlist dict
netlists_dict[Key.TOP2] = (
    netlists_dict[Key.TITLE]
    + netlists_dict[Key.BLANKLINE]
    + netlists_dict[Key.DUT]
    + netlists_dict[Key.LOAD2]
    + netlists_dict[Key.BLANKLINE]
    + netlists_dict[Key.SUPPLIES]
    + netlists_dict[Key.BLANKLINE]
    + netlists_dict[Key.STIMULUS2]
    + netlists_dict[Key.BLANKLINE]
    + netlists_dict[Key.MODELS]
    + netlists_dict[Key.BLANKLINE]
    + netlists_dict[Key.CONTROL2]
    + netlists_dict[Key.END_LINE]
)
# write netlist to a file so ngspice can read it
top2_filename: Path = paths_dict[Key.NETLISTS_PATH] / "top2.cir"
netlists_dict[Key.TOP2].write_to_file(top2_filename)

# prepare simulate object, print out command, and simulate
sim2: spi.Simulate = spi.Simulate(
    paths_dict[Key.NGSPICE_EXE],
    top1_filename,
    paths_dict[Key.SIM_TRANSCRIPT_FILENAME],
    "sim2",
)
# spi.print_section("Ngspice Command", sim1) # print out command
sim2.run()  # run the Ngspice simulation

# convert the raw results into list of SimResults objects
# sim_results2: list[spi.SimResults] = [
#     spi.SimResults.from_file(analysis.cmd_type, analysis.results_filename)
#     for analysis in list_of_analyses
# ]

# give each SimResults object a more descriptive name
# tr1_results = sim_results


In [ ]:
def create_control_section(list_of_analyses: list[spi.Analyses]) -> spi.Netlist:

    my_control = spi.Control()  # create 'my_control' object
    # my_control.insert_lines(["listing"])  # cmd to list out netlist
    for analysis in list_of_analyses:  # statements for all analyses
        my_control.insert_lines(analysis.lines_for_cntl())

    return spi.Netlist(str(my_control))  # create netlist object

In [ ]:
def execute_ngspice(
    ngspice_exe: Path, netlist: Path, sim_transcript: Path, sim_name: str
) -> None:
    """Execute ngspice"""

    # prepare simulate object, print out command, and simulate
    sim1: spi.Simulate = spi.Simulate(ngspice_exe, netlist, sim_transcript, sim_name)
    # spi.print_section("Ngspice Command", sim1) # print out command
    sim1.run()  # run the Ngspice simulation

In [ ]:
def convert_to_numpy(list_of_analyses: list[spi.Analyses]) -> list[spi.SimResults]:
    # convert the raw results into list of SimResults objects
    return [
        spi.SimResults.from_file(analysis.cmd_type, analysis.results_filename)
        for analysis in list_of_analyses
    ]

In [ ]:
def define_analyses2(
    paths_dict: dict[str, Path], vectors_dict: dict[str, spi.Vectors]
) -> list[spi.Analyses]:
    """Define and return a list of analyses"""

    # vectors for each analysis and path to put results
    vec_all: spi.Vectors = vectors_dict[Key.VEC_ALL]
    results_path: Path = paths_dict[Key.RESULTS_PATH]

    # create empty list. Next sections define
    list_of_analyses: list[spi.Analyses] = []

    # 1st analysis
    tr_cmd = "tran 1e-9 20e-6"
    tr1 = spi.Analyses("tr1", "tran", tr_cmd, vec_all, results_path)
    list_of_analyses.append(tr1)

    return list_of_analyses

In [ ]:
def create_top2_netlist(
    netlists_dict: dict[str, spi.Netlist]
) -> dict[str, spi.Netlist]:
    """Create netlist object and add to netlist dictionary"""

    # concatenate all tne netlists to make top1 and add to netlist dict
    netlists_dict[Key.TOP2] = (
        netlists_dict[Key.TITLE]
        + netlists_dict[Key.BLANKLINE]
        + netlists_dict[Key.DUT]
        + netlists_dict[Key.LOAD2]
        + netlists_dict[Key.BLANKLINE]
        + netlists_dict[Key.SUPPLIES]
        + netlists_dict[Key.BLANKLINE]
        + netlists_dict[Key.STIMULUS2]
        + netlists_dict[Key.BLANKLINE]
        + netlists_dict[Key.MODELS]
        + netlists_dict[Key.BLANKLINE]
        + netlists_dict[Key.CONTROL2]
        + netlists_dict[Key.END_LINE]
    )
    return netlists_dict

In [ ]:
def simulate2(
    paths_dict: dict[str, Path],
    netlists_dict: dict[str, spi.Netlist],
    vectors_dict: dict[str, spi.Vectors],
) -> tuple[list[spi.SimResults], dict[str, spi.Netlist]]:
    """Setup and run the first simulation. We will be doing several things in sequence:
    1. Define analyses
    2. Create control section
    3. Create top2 netlist
    4. Write top2 netlist to file
    5. Execute ngspice
    6. Convert raw results to numpy arrays
    """
    list_of_analyses = define_analyses2(paths_dict, vectors_dict)  # step 1
    netlists_dict[Key.CONTROL2] = create_control_section(list_of_analyses)  # step 2
    netlists_dict = create_top2_netlist(netlists_dict)  # step 3
    top2_filename: Path = paths_dict[Key.NETLISTS_PATH] / "top2.cir"  # step 4
    netlists_dict[Key.TOP2].write_to_file(top2_filename)
    # step 5
    execute_ngspice(
        paths_dict[Key.NGSPICE_EXE],
        top2_filename,
        paths_dict[Key.SIM_TRANSCRIPT_FILENAME],
        "sim2",
    )
    sim_results: list[spi.SimResults] = convert_to_numpy(list_of_analyses)  # step 6
    return sim_results, netlists_dict

In [ ]:
def plot_tr(
    sim_result: spi.SimResults, vectors_to_plot: spi.Vectors, results_path: Path
) -> None:
    """plot tranisent results"""
    plot_data = sim_result.x_axis_and_sigs(vectors_to_plot.list_out())
    y_names = vectors_to_plot.list_out()
    my_plt = spi.Plot("tr_plt", plot_data, y_names, results_path)
    my_plt.set_title("sim2 transient results")
    my_plt.define_axes(("time", "sec", "linear"), ("voltage", "V", "linear"))
    my_plt.png()    # create png file and send to results directory
    spi.display_plots()

In [ ]:
def analyze_results(
    sim_results1: list[spi.SimResults],
    sim_results2: list[spi.SimResults],
    vectors_dict: dict[str, spi.Vectors],
    results_path: Path,
) -> None:
    # give each SimResults object a more descriptive name
    op1_results, tf1_results = sim_results1
    tr1: spi.SimResults = sim_results2[0]
    vec_out: spi.Vectors = vectors_dict[Key.VEC_OUT]  # easier name

    # diaplay results for operating point analysis
    spi.print_section("Operating Point Results", op1_results.print_table())

    # display results for small signal transfer function analysis
    spi.print_section("Part 1: Transfer Function Results", tf1_results.print_table())

    tr2 = copy.deepcopy(tr1)
    tr2.vec_subset(vec_out.list_out())  # limit to just "out" signal
    tr2.x_range(9e-6, 12e-6, 1000)  # limit range to just step results
    plot_tr(tr2, vec_out, results_path)

    tr2_numpys: list[numpy_flt] = tr2.x_axis_and_sigs(vec_out.list_out())
    my_meas:spi.StepInfo = spi.StepInfo(tr2_numpys[0], tr2_numpys[1], 9e-6, 12e-6, 10000)
    vin_delta:float = 500.0 - 15.0
    vout_delta:float = my_meas.ydelta
    a_s_ol:float = vout_delta / vin_delta 
    stuff:str = f"Open loop gain (DC audio susceptibility): {a_s_ol:.3g}"
    spi.print_section("Part 2 calculations", stuff)

In [ ]:
# Simulate2: Transient
sim_results2, netlists_dict = simulate2(paths_dict, netlists_dict, vectors_dict)

analyze_results(sim_results1, sim_results2, vectors_dict, paths_dict[Key.RESULTS_PATH])